# STEP18 メモリ使用量をへらすモード

- STEP1:これまでDeZeroの変数と関数を作った
- STEP2:関数としてSquareを作った
- STEP3:別の新しい関数を実装し複数の関数を組み合わせて計算を行う
- STEP4:数値微分でいったん微分を計算してみる
- STEP5:バックプロパゲーションの仕組み
- STEP6:VariableとFunctionクラスを拡張して、バックプロパゲーションを用いて微分を求められるように実装
- STEP7:順伝搬がどのような計算であっても自動的に逆伝搬を計算できるようにする, 具体的にはVariableクラスを拡張し使用した関数情報を保持できるようにする
- STEP8:処理効率の改善するために、backwardメソッドをwhileループに置き換える。Variable関数のみの書き換えでOK
- STEP9:pythonの関数として使えるようにする, y.grad=np.array(1.0)を省略する, ndarrayだけ扱う
- STEP10:DeepLearningのフレームワークのテスト方法について説明
- STEP11:関数に対して可変長入出力に対応する
- STEP12:11の拡張
- STEP13:逆伝搬に関しても関数に対して可変長入出力に対応する
- STEP14:同じ変数を繰り返し使うと発生する問題に対応する y = add(x, x)
- STEP15:さまざまなトポロジーの計算グラフに対応すること
- STEP16:さまざまなトポロジーの計算グラフに対応すること
- STEP17:パフォーマンス改善テクニック: pythonのメモリ管理について学ぶ
- STEP18:メモリ削減テクニックとして不要な微分値は保持しない方法の導入, 逆伝搬の必要がないモードを準備

In [1]:
#DIR = "deep-learning-from-scratch-3/steps/"
#! diff $DIR/step17.py $DIR/step18.py -y

## 各種コンポーネント

In [2]:
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

## Test: 不要な微分は保持しない

入力層以外の微分値は値を保持していないことを確認する

![](https://docs.google.com/drawings/d/e/2PACX-1vSmNihw0cI_PwaKQpXYzcqBnMYtY5huSQkua6_Q3-vkHQDHCuf8Vr6kb0WD3dgw7crH9_5Q20IcRzer/pub?w=934&h=428)

In [3]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

In [4]:
# 途中の層
print(y.grad, t.grad)    # None None
# 入力層
print(x0.grad, x1.grad)  # 2.0 1.0

None None
2.0 1.0


## Test: with 文により逆伝搬なしモードに切り替える

バックプロパゲーションが不要なとき、メモリを節約する

追加したコードを再掲

```python
class Config:
    enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config('enable_backprop', False)
```

In [5]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

In [6]:
#y.backward()
print(y.generation)

0


上記のように記述するのが面倒なときは下記でもよい。同じ意味

In [7]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

# 付録1

## [DeZero] with文による切り替え

自分で `f.close` をちゃんと書く必要あり。めんどくさい

In [8]:
f = open('sample.txt', 'w')
f.write('hello world')
f.close

<function TextIOWrapper.close()>

`with` 文を使うとこの手間を省ける

In [9]:
with open('sample.txt', 'w') as f:
    f.write('hello world')

この`with` 文の仕組みを使って「逆伝搬無効モード」へと切り替えることを考える

## [DeZero] withを使ったモード切り替えのサンプル, やりたいことは with+openのような前処理/後処理が可能なwith+<自作関数>

参考: https://qiita.com/QUANON/items/c5868b6c65f8062f5876

通常, try, exept等を使って例外処理を行うが、ここでは、withを使ったモード切り替えに応用する

- 例外処理の基本: try, except(今回は使わない)
- 終了時に常に行う処理: finally


In [10]:
import contextlib
@contextlib.contextmanager
def config_test():
    print('start')    # 前処理
    try:
        yield
    finally:
        print('done') # 後処理

with config_test():
    print('process...')

start
process...
done


ちなみに、以下のようにも書ける

In [35]:
class config_test():

    def __enter__(self):
        print('start')    # 前処理

    def __exit__(self, type, value, traceback):
        print('done') # 後処理

with config_test():
    print('process...')

start
process...
done


# 付録2 Pythonプログラミングの補足

## with 文と @contextlib.contextmanager が便利

https://qiita.com/QUANON/items/c5868b6c65f8062f5876

### `try/finally` を使って必ず、ファイルがcloseされることを保証するコード

0割でわざとエラーを出しているが、必ずfinallyを通ってファイルがcloseされる

In [12]:
def divide_by_zero(n):
    return n / 0

f = None
try:
    f = open('sample.txt', 'r')
    divide_by_zero(len(f.read()))
finally:
    if f:
        f.close()

ZeroDivisionError: division by zero

以下でファイルがクローズされたことを確認できる

In [13]:
print(f.closed)

True


### 上記をwith文を使って書き換えてみる

In [14]:
with open('hidamari.txt', 'r') as f:
    divide_by_zero(len(f.read()))

FileNotFoundError: [Errno 2] No such file or directory: 'hidamari.txt'

In [15]:
print(f.closed)

True


### with+open を with+<自作関数> でもやってみたい。どうすればよいか？

まずはopenを自作してみて理解する。

- クラス中で `__enter__` メソッド、 `__exit__` メソッドを定義すると、`with` 構文に対応する独自クラスを作成できます。
- 一般には `__enter__` メソッドでリソースを確保するような処理、`__exit__` メソッドでリソースを解放するような処理を実装します。

このwith文に渡されるオブジェクトをコンテキストマネージャと呼びます。

以下の with文の `f` にはコンテキストマネージャの `__enter__` の返り値がバインドされます。
そして withブロックを抜けたときに `__exit__` が呼び出されるという仕組みです。__enter__ が前処理、__exit__ が後処理のイメージ

https://techacademy.jp/magazine/31663

In [16]:
class Reading:
    def __init__(self, filename):
        self.filename = filename

    def __enter__(self):
        self.file = open(self.filename, 'r')

        return self.file

    def __exit__(self, type, value, traceback):
        self.file.close()

In [17]:
with Reading('sample.txt') as f:
    divide_by_zero(len(f.read()))

ZeroDivisionError: division by zero

In [18]:
print(f.closed)

True


### 毎回 __enter__, __exit__を書くのは大変, `@contextlib.contextmanager` で同様のことができる

- `with` ブロック内で発生した例外は `yield` を実行している箇所で再送出されます。
- `with` ブロック内でユーザがどんなコードを実行するのかはコンテキストマネージャからは予想できません。
- そのため `yield` は `try/finally` で囲っておく必要があります。

In [19]:
import contextlib

@contextlib.contextmanager
def reading(filename):
    file = open(filename, 'r')
    try:
        yield file
    finally:
        file.close()

In [20]:
with reading('sample.txt') as f:
    print(f.read())

hello world


In [21]:
print(f.closed)

True


## yield

### yieldとは

- https://www.sejuku.net/blog/23716
- http://ailaby.com/yield/

関数を一時的に実行停止させることが出来る機能を持つ文である


### yieldを使うメリット

例えば、return文でそのまま値を返す関数を作ったとします。一度に大きなリストが返ってくるような関数だと、たくさんのメモリを一度に消費してしまうことになります。

そのようなときは、yieldを使う事でその莫大な量の戻り値を小分けにして返すことが出来ます。これによって関数の実行をその都度中断し、少量ずつデータを返す事でメモリの消費量を抑えることが出来るようになります。

### yieldの基本的な使い方

In [22]:
def myfunc():
    yield 'one'
    yield 'two'
    yield 'three'
for x in myfunc():
    print(x)

one
two
three


In [23]:
x = myfunc()

In [24]:
x.__next__()

'one'

In [25]:
x.__next__()

'two'

In [26]:
x.__next__()

'three'

In [27]:
def myfunc2(x:int):
    # 0からxまでの値を2倍して返す
    for x_ in range(x):
        yield x_*2
        
for i in myfunc2(5):
    print(i)

0
2
4
6
8


In [28]:
def myfunc():
    yield 'one'
    yield 'two'
    yield 'three'

generator = myfunc()
 
print(next(generator))
print(next(generator))
print(next(generator))

one
two
three


## getattr()

https://techacademy.jp/magazine/31147

- オブジェクトで指定された属性を呼び出す関数です。
    - 属性とは、クラスのメンバ変数とメソッドのことを指します。
- getattr()関数を使用することで..
    - オブジェクトのメンバ変数の値を取得することができます。
    - オブジェクトのメソッドを呼び出すことができます。



In [29]:
class hello():
    def __init__(self):
        self.x = 1
        self.y = 2
    def plus(self):
        return self.x + self.y
    def minus(self):
        return self.y - self.x
    def args(self, s):
        return "I got {}".format(s)

cl = hello() 

In [30]:
print(getattr(cl, "x"))
print(getattr(cl, "plus")())
print(getattr(cl, "minus")())
print(getattr(cl, "multi", "None"))
print(getattr(cl, "args")("Hello World"))

1
3
1
None
I got Hello World


## setattr()

In [31]:
class User():
    pass
 
obj = User()
setattr(obj, "name", "Kuro")

In [32]:
print(obj.name)

Kuro


In [33]:
class User():
    def __init__(self, attrs):
        for k, v in attrs.items():
            setattr(self, k, v)
 
obj = User({"name": "Kuro", "age": 30, "memo": "very cool!"})

In [34]:
print(obj.name)
# Kuro
print(obj.age)
# 30
print(obj.memo)
# very cool!

Kuro
30
very cool!
